# 树上的操作

**题目来源：力扣（LeetCode）**

**链接：https://leetcode-cn.com/problems/operations-on-tree/**

## 一、题目

给你一棵 n 个节点的树，编号从 0 到 n - 1 ，以父节点数组 parent 的形式给出，其中 parent[i] 是第 i 个节点的父节点。树的根节点为 0 号节点，所以 parent[0] = -1 ，因为它没有父节点。你想要设计一个数据结构实现树里面对节点的加锁，解锁和升级操作。

数据结构需要支持如下函数：

- Lock：指定用户给指定节点 上锁 ，上锁后其他用户将无法给同一节点上锁。只有当节点处于未上锁的状态下，才能进行上锁操作。
- Unlock：指定用户给指定节点 解锁 ，只有当指定节点当前正被指定用户锁住时，才能执行该解锁操作。
- Upgrade：指定用户给指定节点 上锁 ，并且将该节点的所有子孙节点 解锁 。只有如下 3 个条件 全部 满足时才能执行升级操作：
    - 指定节点当前状态为未上锁。
    - 指定节点至少有一个上锁状态的子孙节点（可以是 任意 用户上锁的）。
    - 指定节点没有任何上锁的祖先节点。

请你实现 LockingTree 类：

- LockingTree(int[] parent) 用父节点数组初始化数据结构。
- lock(int num, int user) 如果 id 为 user 的用户可以给节点 num 上锁，那么返回 true ，否则返回 false 。如果可以执行此操作，节点 num 会被 id 为 user 的用户 上锁 。
- unlock(int num, int user) 如果 id 为 user 的用户可以给节点 num 解锁，那么返回 true ，否则返回 false 。如果可以执行此操作，节点 num 变为 未上锁 状态。
- upgrade(int num, int user) 如果 id 为 user 的用户可以给节点 num 升级，那么返回 true ，否则返回 false 。如果可以执行此操作，节点 num 会被升级 。


## 二、示例

输入：

["LockingTree", "lock", "unlock", "unlock", "lock", "upgrade", "lock"]

[[[-1, 0, 0, 1, 1, 2, 2]], [2, 2], [2, 3], [2, 2], [4, 5], [0, 1], [0, 1]]

输出：

[null, true, false, true, true, true, false]

解释：

```
lockingTree = LockingTree([-1, 0, 0, 1, 1, 2, 2])
lockingTree.lock(2, 2)    # 返回 true ，因为节点 2 未上锁。
                          # 节点 2 被用户 2 上锁。
lockingTree.unlock(2, 3)  # 返回 false ，因为用户 3 无法解锁被用户 2 上锁的节点。
lockingTree.unlock(2, 2)  # 返回 true ，因为节点 2 之前被用户 2 上锁。
                          # 节点 2 现在变为未上锁状态。
lockingTree.lock(4, 5)    # 返回 true ，因为节点 4 未上锁。
                          # 节点 4 被用户 5 上锁。
lockingTree.upgrade(0, 1) # 返回 true ，因为节点 0 未上锁且至少有一个被上锁的子孙节点（节点 4）。
                          # 节点 0 被用户 1 上锁，节点 4 变为未上锁。
lockingTree.lock(0, 1)    # 返回 false ，因为节点 0 已经被上锁了。
```

## 三、解题思路

### 利用map记忆节点关系

In [1]:
class LockingTree:

    def __init__(self, parent):
        self._descendant = {}
        self._lockStatus = {}
        self._ancestors = {}
        for node in range(len(parent)):
            self._lockStatus[node] = -1
            if parent[node] != -1:
                self._ancestors[node] = [parent[node]]
        for key in self._ancestors.keys():
            if self._ancestors[key]:
                parent = self._ancestors[key][0]
                while parent in self._ancestors and self._ancestors[parent]:
                    self._ancestors[key].append(self._ancestors[parent][0])
                    parent = self._ancestors[parent][0]
        for key in self._ancestors.keys():
            ancestor = self._ancestors[key]
            for a in ancestor:
                if a in self._descendant:
                    self._descendant[a].append(key)
                else:
                    self._descendant[a] = [key]
        for key in self._ancestors.keys():
            self._ancestors[key] = set(self._ancestors[key])
        for key in self._descendant.keys():
            self._descendant[key] = set(self._descendant[key])

    def lock(self, num: int, user: int) -> bool:
        if self._lockStatus[num] == -1:
            self._lockStatus[num] = user
            return True
        return False

    def unlock(self, num: int, user: int) -> bool:
        if self._lockStatus[num] == user:
            self._lockStatus[num] = -1
            return True
        return False

    def upgrade(self, num: int, user: int) -> bool:
        if self._lockStatus[num] != -1:
            return False
        if num in self._ancestors:
            for a in self._ancestors[num]:
                if self._lockStatus[a] != -1:
                    return False
        childHasLock = False
        needUnlockIfUpgrade = set()
        if num not in self._descendant:
            return False
        for c in self._descendant[num]:
            if self._lockStatus[c] != -1:
                needUnlockIfUpgrade.add(c)
                childHasLock = True
        if not childHasLock:
            return False
        for node in needUnlockIfUpgrade:
            self._lockStatus[node] = -1
        self._lockStatus[num] = user
        return True

In [2]:
lockingTree = LockingTree([-1, 0, 0, 1, 1, 2, 2])
lockingTree.lock(2, 2)    # 返回 true ，因为节点 2 未上锁。节点 2 被用户 2 上锁。

True

In [3]:
lockingTree.unlock(2, 3)  # 返回 false ，因为用户 3 无法解锁被用户 2 上锁的节点。

False

In [4]:
lockingTree.unlock(2, 2)  # 返回 true ，因为节点 2 之前被用户 2 上锁。节点 2 现在变为未上锁状态。

True

In [5]:
lockingTree.lock(4, 5)    # 返回 true ，因为节点 4 未上锁。
                          # 节点 4 被用户 5 上锁。

True

In [6]:
lockingTree.upgrade(0, 1) # 返回 true ，因为节点 0 未上锁且至少有一个被上锁的子孙节点（节点 4）。
                          # 节点 0 被用户 1 上锁，节点 4 变为未上锁。

True

In [7]:
lockingTree.lock(0, 1)    # 返回 false ，因为节点 0 已经被上锁了。

False